In [1]:
import pandas as pd
from pathlib import Path
from function import load_data

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
files_list_series = pd.Series([i for i in Path("./nav_data").rglob("*") if i.suffix.lower() in {".csv", ".xlsx",".xls"}])

In [4]:
fund_info = load_data("产品代码.xlsx")
fund_info["基金代码"] = fund_info["基金代码"].astype(str)

In [7]:
fund_info

,大类策略,基金代码,基金名称,基准代码,基准名称,最大回撤阈值
0,灵活配置,SGV993,聚鸣涌金8号A期,881001.WI,万得全A,-0.1
1,灵活配置,SGJ508,玄元元丰2号A,881001.WI,万得全A,-0.1
2,灵活配置,SLY951,山楂树五期,881001.WI,万得全A,-0.1
3,灵活配置,SSS087,元品南星A,881001.WI,万得全A,-0.1
4,灵活配置,SLZ483,循远安心十三号A,881001.WI,万得全A,-0.1
...,...,...,...,...,...,...
84,多策略,SVC372,博普CTA股票全市场五号1期,881001.WI,万得全A,-0.1
85,多策略,953832,君享大类资产全天候2号,881001.WI,万得全A,-0.1
86,其他,SSB167,天演天睿,881001.WI,万得全A,-0.1
87,其他,STU233,博普凤祥13号,881001.WI,万得全A,-0.1


In [8]:
nav_dfs = pd.DataFrame()
for row in fund_info.itertuples(index=False, name=None):
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[1] in x.stem)]
    nav_df = load_data(nav_df_path.iloc[0])
    nav_df = nav_df.rename(columns={'基金id': '基金名称', '累计单位净值': '复权净值'})
    nav_df["基金代码"] = row[1]
    nav_df = nav_df[["基金代码","基金名称","日期","单位净值","累计净值","复权净值"]]
    nav_dfs = pd.concat([nav_dfs, nav_df], ignore_index=True)

In [9]:
nav_dfs.to_csv("nav_dfs.csv",index=False)

In [11]:
nav_dfs = load_data("nav_dfs.csv")

In [14]:
nav_dfs["基金代码"].unique()

array(['SGV993', 'SGJ508', 'SLY951', 'SSS087', 'SLZ483', 'SSJ285',
       'SAJH67', 'NM798C', 'SLY430', 'STD157', 'SVM265', 'SXG650',
       'STM350', 'XH489B', 'SNV630', 'SNL974', 'SJQ355', 'STA888',
       'SACZ78', 'SQW547', 'SNS136', 'SND877', 'SGA864', 'SQJ763',
       'SSB692', 'B0537A', 'SGN468', 'S25534', 'SQD495', 'SY2054',
       'SSR924', 'SNK918', 'SB2582', 'SCQ830', 'STR328', 'SVG973',
       'SVG083', 'XM601B', 'SAAB63', 'SJG631', 'SJY709', 'STL559',
       'SNS444', 'SQD760', 'SQF469', 'SVY454', 'SQB871', 'VZ606B',
       'SQT558', 'SLC039', 'SAGF01', 'SXJ577', 'SAHV07', 'SZZ812',
       'B7800B', 'SST242', 'SZX403', 'SACZ89', 'ZZ676C', 'SGK063',
       'SQW111', 'SSG488', 'SAFE06', 'SZF629', 'SQB775', 'SJJ733',
       'SVP717', 'SZH788', 'SXY216', '953968', 'SAFP48', 'SAGF97',
       'TB284B', 'SZF187', 'AHS76B', 'ACZ97B', 'TW548B', 'SVP574',
       'SAFG55', 'SZG938', 'STR499', 'SLZ237', 'SVE049', 'STJ882',
       'SVC372', '953832', 'SSB167', 'STU233', 'SXA265'], dtyp

In [ ]:
from function import load_data
from nav_research import NavResearch
import pandas as pd
from pathlib import Path

basic_info = load_data("产品目录.xlsx")
# strategy_info = basic_info[basic_info["大类策略"] == "量化CTA"]


def multi_fund_comparison(tables, fund_name):
    data1 = tables[0]
    data2 = tables[1].head(1)
    data3 = pd.DataFrame()
    data4 = tables[2]
    years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]
    for year in years:
        data3[f"{year}收益"] = data4.loc[
            data4["分年度业绩"] == year, f"{fund_name}_收益"
        ].values
        data3[f"{year}最大回撤"] = data4.loc[
            data4["分年度业绩"] == year, f"{fund_name}_最大回撤"
        ].values
    data = pd.concat([data1, data2, data3], axis=1)
    data.drop(columns=["基准指数", "整体业绩"], inplace=True)
    return data


data = pd.DataFrame()
files_list_series = pd.Series(
    [i for i in Path("./data").rglob("*") if i.suffix.lower() in {".csv", ".xlsx",".xls"}]
)
for row in basic_info.itertuples(index=False, name=None):
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[3] in x.stem)]
    assert len(nav_df_path) == 1, "找到多个文件或者没有稳健"
    demo = NavResearch(nav_df_path.item(), row[0], row[3], row[4], row[5], row[6])
    demo.get_data()
    tables = demo.get_analysis_table()
    nav_df = multi_fund_comparison(tables, row[3])
    data = pd.concat([data, nav_df], axis=0)

data.to_excel("data.xlsx", index=False)